In [1]:
# Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License").
# You may not use this file except in compliance with the License.
# A copy of the License is located at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# or in the "license" file accompanying this file. This file is distributed
# on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
# express or implied. See the License for the specific language governing
# permissions and limitations under the License.

"""
This example shows how to fit a model and evaluate its predictions.
"""
import pprint
from functools import partial

import pandas as pd

from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.trainer import Trainer

datasets = [
    "m4_hourly",
    "m4_daily",
    "m4_weekly",
    "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

epochs = 100
num_batches_per_epoch = 50

estimators = [
    partial(
        SimpleFeedForwardEstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
    partial(
        DeepAREstimator,
        distr_output=PiecewiseLinearOutput(8),
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
    partial(
        MQCNNEstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
    )

    print(f"evaluating {estimator} on {dataset}")

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())


INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.


evaluating gluonts.model.simple_feedforward._estimator.SimpleFeedForwardEstimator(batch_normalization=False, context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), freq="H", mean_scaling=True, num_hidden_dimensions=[40, 40], num_parallel_samples=100, prediction_length=48, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=100, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08)) on TrainDatasets(metadata=<MetaData freq='H' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='414'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=48>, train=<gluonts.dataset.common.FileDataset object at 0x0000023F27D22DD8>, test=<gluonts.dataset.common.FileDataset object at 0x0000023F27D351D0>)


INFO:root:Start model training
INFO:root:Number of parameters in SimpleFeedForwardTrainingNetwork: 1963
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 41.73it/s, avg_epoch_loss=5.49]
INFO:root:Epoch[0] Elapsed time 1.463 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.493589
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 62.70it/s, avg_epoch_loss=4.64]
INFO:root:Epoch[1] Elapsed time 0.807 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.643348
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 61.09it/s, avg_epoch_loss=4.54]
INFO:root:Epoch[2] Elapsed time 0.823 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.539621
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 56.98it/s, avg_epoch_loss=4.4]
INFO:root:Epoch[3] Elapsed time 0.883 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=4.397228
INFO:root:Epoch[4] Learning rate 

INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=4.359617
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 50/50 [00:00<00:00, 54.62it/s, avg_epoch_loss=4.1]
INFO:root:Epoch[37] Elapsed time 0.924 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=4.099745
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 50/50 [00:00<00:00, 64.06it/s, avg_epoch_loss=3.78]
INFO:root:Epoch[38] Elapsed time 0.793 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=3.775701
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 50/50 [00:01<00:00, 46.63it/s, avg_epoch_loss=4.04]
INFO:root:Epoch[39] Elapsed time 1.080 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=4.042879
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 50/50 [00:01<00:00, 42.08it/s, avg_epoch_loss=4.13]
INFO:root:Epoch[40] Elapsed time 1.205 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=4.126551
INFO:root:Epoch[41] Learning rate is 0.0005
100%|██████████| 

100%|██████████| 50/50 [00:00<00:00, 51.05it/s, avg_epoch_loss=4.11]
INFO:root:Epoch[73] Elapsed time 0.986 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=4.114566
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:00<00:00, 67.07it/s, avg_epoch_loss=4.18]
INFO:root:Epoch[74] Elapsed time 0.752 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=4.182213
INFO:root:Epoch[75] Learning rate is 0.000125
100%|██████████| 50/50 [00:00<00:00, 71.17it/s, avg_epoch_loss=3.87]
INFO:root:Epoch[75] Elapsed time 0.708 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=3.869943
INFO:root:Epoch[76] Learning rate is 0.000125
100%|██████████| 50/50 [00:00<00:00, 67.16it/s, avg_epoch_loss=4.05]
INFO:root:Epoch[76] Elapsed time 0.751 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=4.051212
INFO:root:Epoch[77] Learning rate is 0.000125
100%|██████████| 50/50 [00:00<00:00, 65.31it/s, avg_epoch_loss=4.1]
INFO:root:Epoch[77] Elapsed time 0.773 secon

...

evaluating gluonts.model.seq2seq._mq_dnn_estimator.MQCNNEstimator(context_length=None, freq="12M", mlp_final_dim=20, mlp_hidden_dimension_seq=[], prediction_length=6, quantiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=100, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08)) on TrainDatasets(metadata=<MetaData freq='12M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='23000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=6>, train=<gluonts.dataset.common.FileDataset object at 0x0000023F33117C88>, test=<gluonts.dataset.common.FileDataset object at 0x0000023F35A4DD68>)


INFO:root:Start model training
INFO:root:Number of parameters in ForkingSeq2SeqTrainingNetwork: 219
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 36.65it/s, avg_epoch_loss=453]
INFO:root:Epoch[0] Elapsed time 1.371 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=452.560824
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 37.25it/s, avg_epoch_loss=230]
INFO:root:Epoch[1] Elapsed time 1.348 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=229.838383
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 44.95it/s, avg_epoch_loss=84.9]
INFO:root:Epoch[2] Elapsed time 1.116 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=84.924925
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 48.57it/s, avg_epoch_loss=28.5]
INFO:root:Epoch[3] Elapsed time 1.034 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=28.515454
INFO:root:Epoch[4] Learning rate

100%|██████████| 50/50 [00:01<00:00, 38.54it/s, avg_epoch_loss=77.5]
INFO:root:Epoch[36] Elapsed time 1.303 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=77.534480
INFO:root:Epoch[37] Learning rate is 0.00025
100%|██████████| 50/50 [00:01<00:00, 36.15it/s, avg_epoch_loss=79.7]
INFO:root:Epoch[37] Elapsed time 1.389 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=79.653371
INFO:root:Epoch[38] Learning rate is 0.00025
100%|██████████| 50/50 [00:01<00:00, 37.45it/s, avg_epoch_loss=74.5]
INFO:root:Epoch[38] Elapsed time 1.341 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=74.477691
INFO:root:Epoch[39] Learning rate is 0.00025
100%|██████████| 50/50 [00:01<00:00, 37.87it/s, avg_epoch_loss=59.5]
INFO:root:Epoch[39] Elapsed time 1.326 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=59.538439
INFO:root:Epoch[40] Learning rate is 0.00025
100%|██████████| 50/50 [00:01<00:00, 37.56it/s, avg_epoch_loss=70.3]
INFO:root:Epoch[40] Elapsed time 1.336 seco

100%|██████████| 50/50 [00:01<00:00, 39.00it/s, avg_epoch_loss=67]
INFO:root:Epoch[72] Elapsed time 1.287 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=66.950745
INFO:root:Epoch[73] Learning rate is 5e-05
100%|██████████| 50/50 [00:01<00:00, 35.01it/s, avg_epoch_loss=69.8]
INFO:root:Epoch[73] Elapsed time 1.434 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=69.846764
INFO:root:Epoch[74] Learning rate is 5e-05
100%|██████████| 50/50 [00:01<00:00, 37.93it/s, avg_epoch_loss=58.8]
INFO:root:Epoch[74] Elapsed time 1.323 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=58.842961
INFO:root:Epoch[75] Learning rate is 5e-05
100%|██████████| 50/50 [00:01<00:00, 36.10it/s, avg_epoch_loss=26.4]
INFO:root:Epoch[75] Elapsed time 1.391 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=26.380863
INFO:root:Epoch[76] Learning rate is 5e-05
100%|██████████| 50/50 [00:01<00:00, 38.22it/s, avg_epoch_loss=47.1]
INFO:root:Epoch[76] Elapsed time 1.315 seconds
INFO:r

Cannot extract prediction target since the index of forecast is outside the index of target
Index of forecast: DatetimeIndex(['1752-07-31', '1753-07-31', '1754-07-31', '1755-07-31',
               '1756-07-31', '1757-07-31'],
              dtype='datetime64[ns]', freq='12M')
 Index of target: DatetimeIndex(['1749-12-31', '1750-12-31', '1751-12-31', '1752-12-31',
               '1753-12-31', '1754-12-31', '1755-12-31', '1756-12-31',
               '1757-12-31', '1758-12-31', '1759-12-31', '1760-12-31',
               '1761-12-31', '1762-12-31', '1763-12-31', '1764-12-31',
               '1765-12-31', '1766-12-31', '1767-12-31', '1768-12-31',
               '1769-12-31', '1770-12-31', '1771-12-31', '1772-12-31',
               '1773-12-31', '1774-12-31', '1775-12-31', '1776-12-31',
               '1777-12-31', '1778-12-31', '1779-12-31', '1780-12-31',
               '1781-12-31', '1782-12-31', '1783-12-31', '1784-12-31',
               '1785-12-31'],
              dtype='datetime64[ns]',